In [1]:
def count_para(model):
    trainable_parameters = list(filter(lambda p: p.requires_grad, model.parameters()))
    n_trainable_parameters = sum([p.numel() for p in trainable_parameters])
    return n_trainable_parameters


In [2]:
import os
os.chdir('/home/students/wittmann/jw-dir/repos/conditional_INNs/mnist_cINN')


In [3]:
import config
config.colorize = True
print(config.colorize)

True


In [4]:
import FrEIA
from model import *

In [5]:
print(count_para(model), 'parameters')
print(list(model.modules()).__len__(), 'modules')
print(list(x for x in model.modules() if isinstance(x, FrEIA.modules.coupling_layers._BaseCouplingBlock)).__len__(), 'coupling layers')
# 3 conv conditioned, 7 fully not condiditoned
print(count_para(list(x for x in model.modules() if isinstance(x, FrEIA.modules.coupling_layers._BaseCouplingBlock))[0]), 'parameters conv')
print(count_para(list(x for x in model.modules() if isinstance(x, FrEIA.modules.coupling_layers._BaseCouplingBlock))[-1]), 'parameters fully')

print(list(count_para(x) for x in list(x for x in model.modules() if isinstance(x, FrEIA.modules.coupling_layers._BaseCouplingBlock))))

# print(count_para(cinn.cinn), 'parameters')
print((model.dims_in), 'input')
print((model.dims_c), 'cond')


13681884 parameters
159 modules
10 coupling layers
26644 parameters conv
1943136 parameters fully
[26644, 26644, 26644, 1943136, 1943136, 1943136, 1943136, 1943136, 1943136, 1943136]
[(3, 28, 28)] input
[(65, 28, 28)] cond


In [6]:
model(torch.rand(7,3,28,28), torch.rand(7,65,28,28))[0].shape

torch.Size([7, 2352])

In [7]:
import cond_net

In [8]:
net = cond_net.Net()
print(count_para(net), 'parameters')


913610 parameters


In [9]:
# wtf 512 fixed batchsize???
print(net(torch.rand(512,1,28,28)).shape)
print(net.features(torch.rand(512,1,28,28)).shape)


torch.Size([512, 10])
torch.Size([512, 64])


In [9]:
list(x for x in model.modules() if isinstance(x, FrEIA.modules.coupling_layers._BaseCouplingBlock))[0]
# why 4 output for subnet 2? -> 2channels, 2x shift + 2x scale
# why 2 output for subnet 1? -> 1channels, 1x shift + 1x scale


GLOWCouplingBlock(
  (subnet1): F_fully_conv(
    (conv1): Conv2d(67, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv2): Conv2d(131, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv3): Conv2d(195, 2, kernel_size=(1, 1), stride=(1, 1))
  )
  (subnet2): F_fully_conv(
    (conv1): Conv2d(66, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv2): Conv2d(130, 64, kernel_size=(1, 1), stride=(1, 1))
    (conv3): Conv2d(194, 4, kernel_size=(1, 1), stride=(1, 1))
  )
)

In [13]:
list(x for x in model.modules() if isinstance(x, FrEIA.modules.coupling_layers._BaseCouplingBlock))[-1]

GLOWCouplingBlock(
  (subnet1): F_fully_shallow(
    (d1): Dropout(p=0.0, inplace=False)
    (d2): Dropout(p=0.0, inplace=False)
    (fc1): Linear(in_features=1176, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=256, bias=True)
    (fc3): Linear(in_features=256, out_features=2352, bias=True)
    (nl1): LeakyReLU(negative_slope=0.01)
    (nl2): LeakyReLU(negative_slope=0.01)
  )
  (subnet2): F_fully_shallow(
    (d1): Dropout(p=0.0, inplace=False)
    (d2): Dropout(p=0.0, inplace=False)
    (fc1): Linear(in_features=1176, out_features=256, bias=True)
    (fc2): Linear(in_features=256, out_features=256, bias=True)
    (fc3): Linear(in_features=256, out_features=2352, bias=True)
    (nl1): LeakyReLU(negative_slope=0.01)
    (nl2): LeakyReLU(negative_slope=0.01)
  )
)

In [10]:
for x in model.node_list_fwd:
    print(x, x.conditions, x.condition_dims)

InputNode 'inp': [] ->  -> [(3, 28, 28)] [] []
Node 'permute_0': [(3, 28, 28)] -> PermuteRandom -> [(3, 28, 28)] [] []
ConditionNode '263350': [] ->  -> [(65, 28, 28)] [] []
Node 'conv_0': [(3, 28, 28)] -> GLOWCouplingBlock -> [(3, 28, 28)] [(ConditionNode '263350', 0)] [(65, 28, 28)]
Node 'permute_1': [(3, 28, 28)] -> PermuteRandom -> [(3, 28, 28)] [] []
Node 'conv_1': [(3, 28, 28)] -> GLOWCouplingBlock -> [(3, 28, 28)] [(ConditionNode '263350', 0)] [(65, 28, 28)]
Node 'permute_2': [(3, 28, 28)] -> PermuteRandom -> [(3, 28, 28)] [] []
Node 'conv_2': [(3, 28, 28)] -> GLOWCouplingBlock -> [(3, 28, 28)] [(ConditionNode '263350', 0)] [(65, 28, 28)]
Node 'flatten': [(3, 28, 28)] -> Flatten -> ((2352,),) [] []
Node 'permute_0': [(2352,)] -> PermuteRandom -> [(2352,)] [] []
Node 'fc_0': [(2352,)] -> GLOWCouplingBlock -> [(2352,)] [] []
Node 'permute_1': [(2352,)] -> PermuteRandom -> [(2352,)] [] []
Node 'fc_1': [(2352,)] -> GLOWCouplingBlock -> [(2352,)] [] []
Node 'permute_2': [(2352,)] -> 

In [11]:
model

ReversibleGraphNet(
  (module_list): ModuleList(
    (0): PermuteRandom()
    (1): GLOWCouplingBlock(
      (subnet1): F_fully_conv(
        (conv1): Conv2d(67, 64, kernel_size=(1, 1), stride=(1, 1))
        (conv2): Conv2d(131, 64, kernel_size=(1, 1), stride=(1, 1))
        (conv3): Conv2d(195, 2, kernel_size=(1, 1), stride=(1, 1))
      )
      (subnet2): F_fully_conv(
        (conv1): Conv2d(66, 64, kernel_size=(1, 1), stride=(1, 1))
        (conv2): Conv2d(130, 64, kernel_size=(1, 1), stride=(1, 1))
        (conv3): Conv2d(194, 4, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (2): PermuteRandom()
    (3): GLOWCouplingBlock(
      (subnet1): F_fully_conv(
        (conv1): Conv2d(67, 64, kernel_size=(1, 1), stride=(1, 1))
        (conv2): Conv2d(131, 64, kernel_size=(1, 1), stride=(1, 1))
        (conv3): Conv2d(195, 2, kernel_size=(1, 1), stride=(1, 1))
      )
      (subnet2): F_fully_conv(
        (conv1): Conv2d(66, 64, kernel_size=(1, 1), stride=(1, 1))
        (conv2): C